In [1]:
!pip install carla

In [1]:
import carla
import math
import random
import time
import numpy as np
import cv2

## Connecting with Carla

In [2]:
# Run Carla Server
# $ cd /opt/carla-simulator/
# $ ./CarlaUE4.sh

# Connect to the client and get the world object
client = carla.Client('localhost', 2000) 
world = client.get_world() # Access all objects

In [3]:
# Get the blueprint library and the spawn points for the map
bp_lib = world.get_blueprint_library() # access all the blueprint
spawn_points = world.get_map().get_spawn_points() 

## Configuring Weather

In [4]:
weather = carla.WeatherParameters(
    cloudiness=99.0,
    precipitation=30.0,
    sun_altitude_angle=80.0,
    rayleigh_scattering_scale=0,
)

world.set_weather(weather)

## Setting Vehicle

In [5]:
# Get the blueprint for the vehicle you want
vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020') 

# Try spawning the vehicle at a randomly chosen spawn point
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

# Move the spectator behind the vehicle 
spectator = world.get_spectator() 
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)),vehicle.get_transform().rotation) 
spectator.set_transform(transform) 

## Adding other traffics

In [6]:
# Add traffic to the simulation
for i in range(30): 
    vehicle_bp = random.choice(bp_lib.filter('vehicle')) 
    npc = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points)) 

In [7]:
# Set the all vehicles in motion using the Traffic Manager
for v in world.get_actors().filter('*vehicle*'): 
    v.set_autopilot(True)

# Sensors

In [7]:
for bp in bp_lib.filter('camera'):
    print(bp.id)

sensor.camera.depth
sensor.camera.dvs
sensor.camera.optical_flow
sensor.camera.rgb
sensor.camera.semantic_segmentation
sensor.camera.instance_segmentation


## Adding Camera to Vehicle

In [8]:
# Spawn an RGB cammera with an offset from the vehicle center
camera_init_trans = carla.Transform(carla.Location(z=2))

camera_bp = bp_lib.find('sensor.camera.rgb') 
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

# spectator.set_transform(camera.get_transform()) 

In [11]:
# Start the camera saving data to disk
camera.listen(lambda image: image.save_to_disk('out/%06d.png' % image.frame))

In [12]:
# Stop the camera when we've recorded enough data
camera.stop()

In [9]:
# Set Stream Camera Feedback
# RGB Camera
def rgb_camera_callback(image,data_dict):
    data_dict['rgb_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

In [10]:
# Show Stream RGB Camera Feedback
image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'rgb_image': np.zeros((image_h, image_w, 4))}
camera.listen(lambda image: rgb_camera_callback(image, camera_data))

cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['rgb_image'])
cv2.waitKey(1)

while True:
    cv2.imshow('RGB Camera', camera_data['rgb_image'])
    if cv2.waitKey(1)==ord('q'):
        break

cv2.destroyAllWindows()